# Yoga Pose Scoring

## Imports and Constants

In [163]:
import numpy as np
from collections import deque
import json
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks import python
import os
import cv2
import pandas as pd
from cv2 import IMREAD_UNCHANGED
import matplotlib.pyplot as plt

## Map Landmark Mapping

In [164]:
# def compute_angle(a, b, c):
#     """
#     Compute angle ABC (in degrees) given 3 points.
#     a, b, c are numpy arrays of shape (2,) or (3,)
#     """
#     ba = a - b
#     bc = c - b

#     cosine_angle = np.dot(ba, bc) / (
#         np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6
#     )
#     angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
#     return np.degrees(angle)

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle


## Extract Pose Landmarks

In [165]:
# Create a PoseLandmarker object.
base_options = mp.tasks.BaseOptions(
    model_asset_path='pose_landmarker.task'
)
options = mp.tasks.vision.PoseLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.IMAGE,
    output_segmentation_masks=True)
pose_landmarker = mp.tasks.vision.PoseLandmarker.create_from_options(options)

I0000 00:00:1769751231.923845 4413215 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
W0000 00:00:1769751232.056426 4413217 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769751232.075870 4413216 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Pose Normalization

In [166]:
def normalize_landmarks(landmarks):
    left_hip = landmarks[LANDMARKS["left_hip"]]
    right_hip = landmarks[LANDMARKS["right_hip"]]
    hip_center = (left_hip + right_hip) / 2

    left_shoulder = landmarks[LANDMARKS["left_shoulder"]]
    right_shoulder = landmarks[LANDMARKS["right_shoulder"]]
    shoulder_width = np.linalg.norm(left_shoulder - right_shoulder) + 1e-6

    normalized = (landmarks - hip_center) / shoulder_width
    return normalized


## Feature Extraction

In [167]:
def extract_joint_angles(lm):
    return {
        "left_knee": calculate_angle(
            lm[LANDMARKS["left_hip"]],
            lm[LANDMARKS["left_knee"]],
            lm[LANDMARKS["left_ankle"]],
        ),
        "right_knee": calculate_angle(
            lm[LANDMARKS["right_hip"]],
            lm[LANDMARKS["right_knee"]],
            lm[LANDMARKS["right_ankle"]],
        ),
        "left_hip": calculate_angle(
            lm[LANDMARKS["left_shoulder"]],
            lm[LANDMARKS["left_hip"]],
            lm[LANDMARKS["left_knee"]],
        ),
        "right_hip": calculate_angle(
            lm[LANDMARKS["right_shoulder"]],
            lm[LANDMARKS["right_hip"]],
            lm[LANDMARKS["right_knee"]],
        ),
        "left_elbow": calculate_angle(
            lm[LANDMARKS["left_shoulder"]],
            lm[LANDMARKS["left_elbow"]],
            lm[LANDMARKS["left_wrist"]],
        ),
        "right_elbow": calculate_angle(
            lm[LANDMARKS["right_shoulder"]],
            lm[LANDMARKS["right_elbow"]],
            lm[LANDMARKS["right_wrist"]],
        ),
    }


## Reference Pose Detection

In [168]:
LANDMARKS = {
    "left_shoulder": 11,
    "right_shoulder": 12,
    "left_elbow": 13,
    "right_elbow": 14,
    "left_wrist": 15,
    "right_wrist": 16,
    "left_hip": 23,
    "right_hip": 24,
    "left_knee": 25,
    "right_knee": 26,
    "left_ankle": 27,
    "right_ankle": 28
}

ANGLE_TOLERANCE = 15.0

# lm = json.load(open('yoga_landmarks/tree_2.json'))
lm = pd.read_json('yoga_landmarks/tree_2.json')
lm = np.array(lm)
reference_angles = extract_joint_angles(lm) 


## MAE Based Pose Scoring

In [169]:
# def compute_weighted_mae(user_angles, ref_angles, weights):
#     total_error = 0.0

#     for joint, ref_angle in ref_angles.items():
#         if joint not in user_angles:
#             continue
#         error = abs(user_angles[joint] - ref_angle)
#         total_error += weights[joint] * error

#     return total_error


## Convert MAE to Score (0 - 100)

In [170]:
def get_shortest_angle_distance(a, b):
    """Calculates the shortest distance between two angles in degrees."""
    diff = (b - a + 180) % 360 - 180
    return abs(diff)

def mae_to_score(mae, tolerance=ANGLE_TOLERANCE):
    # sigma = tolerance
    # score = np.exp(-0.5 * (mae / sigma) ** 2)
    # return score * 100
    
    if mae <= tolerance:
        return 100.0
    cutoff = 45.0
    score = max(0.0, 1.0 - (mae - tolerance) / (cutoff - tolerance))
    # score = max(0.0, 1.0 - mae / tolerance)
    return score * 100

def compute_mae(user_angles, reference_pose):
    total_error = 0.0
    total_weight = 0.0

    for joint, ref_angle in reference_pose.items():
        if joint not in user_angles:
            continue
        error = get_shortest_angle_distance(user_angles[joint], ref_angle)
        total_error += error
        # total_error += abs(user_angles[joint] - ref_angle)
        total_weight += 1

    return total_error / (total_weight + 1e-6)

## Pose Scoring Pipeline

In [171]:
def load_image(image_path):
    image = cv2.imread(image_path, IMREAD_UNCHANGED)
    if image is None:
        raise ValueError("Image not found")
    return image

In [172]:
def score_pose_from_image(image):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert to MediaPipe Image
    mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=rgb
    )

    # Run pose landmarker
    result = pose_landmarker.detect(mp_image)

    if not result.pose_landmarks:
        return None

    # Extract landmarks (first detected person)
    landmarks = result.pose_landmarks[0]

    # Convert to numpy array (x, y only)
    landmarks_np = np.array(
        [[lm.x, lm.y] for lm in landmarks]
    )

    # Normalize + extract features
    landmarks_np = normalize_landmarks(landmarks_np)
    angles = extract_joint_angles(landmarks_np)

    # Score
    # reference_pose = get_tree_pose_reference(angles)
    # mae = weighted_mae(angles, reference_pose, POSE_WEIGHTS)
    mae = compute_mae(angles, reference_angles)
    score = mae_to_score(mae)


    return {
        "score": round(score, 2),
        "mae": round(mae, 2),
        "angles": angles,
        "reference angles": reference_angles,
        "landmarks": landmarks,
    }


In [173]:
# image = load_image("/Users/weijie/Downloads/archive/Vrksasana/File12.png")
# image = load_image("images/tree_2.jpg")
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [174]:
def show_result(image, show_plot):
    result = score_pose_from_image(image)
    landmark_list = result["landmarks"]

    if show_plot:
        annotated_image = image.copy()
        for idx, landmark in enumerate(landmark_list):
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            cv2.circle(annotated_image, (x, y), 5, (0, 255, 0), -1)

            # Draw connections
            connections = mp.tasks.vision.PoseLandmarksConnections.POSE_LANDMARKS
            for connection in connections:
                start_idx = connection.start
                end_idx = connection.end
                start_point = (int(landmark_list[start_idx].x * image.shape[1]),
                                int(landmark_list[start_idx].y * image.shape[0]))
                end_point = (int(landmark_list[end_idx].x * image.shape[1]),
                                int(landmark_list[end_idx].y * image.shape[0]))
                cv2.line(annotated_image, start_point, end_point, (255, 0, 0), 2)
            
        plt.figure(figsize=(10, 10))
        plt.imshow(annotated_image)
        plt.axis('off')
        plt.show()
    return result

In [193]:
image = load_image("/Users/weijie/Downloads/archive/Vrksasana/File24.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

result = show_result(image, show_plot=False)

for k, v in result.items():
    if k == 'angles' or k == 'reference angles':
        print(f"{k}:")
        for angle_k, angle_v in v.items():
            print(f"\t{angle_k}: {angle_v}")
    else:
        print(f"{k}: {v}")

score: 100.0
mae: 10.83
angles:
	left_knee: 176.78557700680173
	right_knee: 14.79878241404864
	left_hip: 178.25214707723663
	right_hip: 96.95392528194662
	left_elbow: 21.94799571858269
	right_elbow: 21.551002255044317
reference angles:
	left_knee: 177.30304030599115
	right_knee: 38.94845179885842
	left_hip: 175.15963930404658
	right_hip: 128.1149594409297
	left_elbow: 25.71898900904405
	right_elbow: 23.825378119325517
landmarks: [NormalizedLandmark(x=0.5982357263565063, y=0.09445592761039734, z=-0.7386676073074341, visibility=0.9998100399971008, presence=0.9999082088470459, name=None), NormalizedLandmark(x=0.6224892139434814, y=0.07729500532150269, z=-0.652359127998352, visibility=0.999818742275238, presence=0.9997265934944153, name=None), NormalizedLandmark(x=0.6370223164558411, y=0.07721284031867981, z=-0.6524774432182312, visibility=0.9997636675834656, presence=0.9996833801269531, name=None), NormalizedLandmark(x=0.6496479511260986, y=0.07757875323295593, z=-0.6525528430938721, visi

## In Camera Loop

In [176]:
# class StabilityTracker:
#     def __init__(self, window_size=30):
#         self.window_size = window_size
#         self.angle_history = deque(maxlen=window_size)

#     def update(self, angles):
#         self.angle_history.append(list(angles.values()))

#     def stability_score(self):
#         if len(self.angle_history) < 5:
#             return 1.0  # not enough data to penalize

#         variance = np.var(self.angle_history)
#         stability = np.exp(-variance)
#         return stability


In [177]:
# def score_pose(landmarks, stability_tracker=None):
#     # Normalize
#     landmarks = normalize_landmarks(landmarks)
# 
#     # Extract angles
#     user_angles = extract_joint_angles(landmarks)
# 
#     # MAE scoring
#     mae = compute_weighted_mae(
#         user_angles,
#         TREE_POSE_REFERENCE,
#         TREE_POSE_WEIGHTS
#     )
# 
#     base_score = mae_to_score(mae)
# 
#     # Stability bonus
#     if stability_tracker:
#         stability_tracker.update(user_angles)
#         stability = stability_tracker.stability_score()
#         final_score = base_score * stability
#     else:
#         final_score = base_score
# 
#     return {
#         "score": round(final_score, 2),
#         "mae": round(mae, 2),
#         "angles": user_angles
#     }
# 

In [178]:
# stability_tracker = StabilityTracker(window_size=45)
# 
# # landmarks should be a (33, 2) or (33, 3) numpy array from MediaPipe
# # Example:
# # landmarks = np.array([[x0,y0], [x1,y1], ...])
# 
# result = score_pose(landmarks, stability_tracker)
# 
# print("Score:", result["score"])
# print("MAE:", result["mae"])
# 